## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-13-11-04-40 +0000,bbc,Rubio warns Europe of new era in geopolitics b...,https://www.bbc.com/news/articles/c99jxryd9xko...
1,2026-02-13-11-03-00 +0000,wsj,Gold Rally Not Yet at Peak Despite Recent Vola...,https://www.wsj.com/finance/commodities-future...
2,2026-02-13-11-00-42 +0000,wapo,A Black newspaper in the former capital of the...,https://www.washingtonpost.com/business/2026/0...
3,2026-02-13-11-00-34 +0000,wapo,The Don Lemon indictment: Video appears to con...,https://www.washingtonpost.com/investigations/...
4,2026-02-13-11-00-00 +0000,nypost,"Kin of fatal hit-and-run victim, 74, demand j...",https://nypost.com/2026/02/13/us-news/kin-of-f...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2441/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
198,trump,37
208,epstein,21
811,immigration,14
234,ice,14
97,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
208,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...,127
364,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...,87
268,2026-02-12-19-47-08 +0000,bbc,Minnesota immigration enforcement surge is end...,https://www.bbc.com/news/articles/c2lr9w29zwyo...,84
266,2026-02-12-20-01-49 +0000,nyt,Trump Administration to End Surge of Immigrati...,https://www.nytimes.com/2026/02/12/us/minnesot...,84
363,2026-02-12-15-03-32 +0000,cbc,Trump border czar says controversial Minneapol...,https://www.cbc.ca/news/world/us-minnesota-hom...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
208,127,2026-02-12-22-03-00 +0000,wsj,Border czar Tom Homan said the Trump administr...,https://www.wsj.com/politics/policy/tom-homan-...
165,73,2026-02-12-23-36-00 +0000,wsj,"Chris Klomp, the head of Medicare and a former...",https://www.wsj.com/politics/policy/white-hous...
366,67,2026-02-12-14-38-32 +0000,nypost,Woman seen in photo under disgraced ex-Prince ...,https://nypost.com/2026/02/12/us-news/woman-in...
365,56,2026-02-12-14-41-00 +0000,wsj,What a shutdown would mean for the Department ...,https://www.wsj.com/politics/policy/homeland-s...
48,54,2026-02-13-09-24-45 +0000,nypost,Arizona Sheriff Chris Nanos fires back at clai...,https://nypost.com/2026/02/13/us-news/arizona-...
234,41,2026-02-12-21-05-00 +0000,wsj,Judge Blocks Pentagon’s Move to Censure Sen. M...,https://www.wsj.com/us-news/law/judge-blocks-p...
137,41,2026-02-13-01-39-02 +0000,startribune,DOJ moves to drop charges against men arrested...,https://www.startribune.com/doj-moves-to-drop-...
364,40,2026-02-12-15-00-00 +0000,wsj,Josh Gruenbaum’s brash ways as a federal agenc...,https://www.wsj.com/politics/national-security...
280,40,2026-02-12-19-07-22 +0000,latimes,Disney to pay $2.75 million to settle alleged ...,https://www.latimes.com/entertainment-arts/bus...
39,39,2026-02-13-09-59-00 +0000,wsj,"Global Markets, U.S. Futures Fall as Market Wa...",https://www.wsj.com/finance/stocks/global-mark...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
